In [1]:
from selenium.webdriver import Firefox
import re
import pandas as pd
import httpx
import time

In [2]:
driver = Firefox()

In [3]:
driver.get("https://markets.ft.com/research/webservices/securities/v1/docs")

In [4]:
driver.find_element(
    "xpath", "/html/body/div/div[1]/div/div[1]/ul/li[2]/ul/li[7]/a"
).click()

In [5]:
link = driver.find_element(
    "xpath",
    "/html/body/div/div[1]/div/div[2]/article[7]/div[2]/table/tbody/tr/td[2]/div[2]/code",
).text

In [6]:
source = re.search(r"source=(.*)", link)

In [7]:
if source is None:
    raise ValueError("Key not found")
api_key = source.group(1)

In [8]:
driver.close()

In [9]:
securities = [
    ("IE00B3SBSR82:USD", "GMO Quality Investment Fund", "2010-11-10"),
    ("LU0690375182:EUR", "Fundsmith Equity Fund T EUR Acc", "2011-11-02"),
    ("LU0690374615:EUR", "Fundsmith Equity Fund R EUR Acc", "2011-11-02"),
    (
        "IE00B3Z8MM50:GBP",
        "Dimensional World Equity Fund GBP Accumulation",
        "2011-09-09",
    ),
    ("TFTSTIU:FSI", "Straits Times Index USD Gross", "1999-08-31"),
    ("SPXTR:REU", "S&P 500 USD Gross", "1970-01-30"),
    ("SPXNTR:IOM", "S&P 500 USD Net", "1996-03-29"),
    ("TFTAWORLDSU:FSI", "FTSE All-World USD Gross", "1999-12-31"),
    ("_RE2TR:SES", "iEdge S-REIT Leaders SGD Gross", "2010-09-30"),
]

In [ ]:
for symbol, name, start_date in securities:
    response = httpx.get(
        "https://markets.ft.com/research/webservices/securities/v1/historical-series-quotes",
        params={
            "source": api_key,
            "symbols": symbol,
            "dayCount": (pd.Timestamp.today() - pd.Timestamp(start_date)).days,
        },
    )
    backoff = 1
    while (
        response.json()["data"]["items"][0]["historicalSeries"].get(
            "historicalQuoteData"
        )
        is None
    ):
        print(f"Retrying {name}...")
        time.sleep(backoff)
        backoff = min(backoff * 2, 60)
        response = httpx.get(
            "https://markets.ft.com/research/webservices/securities/v1/historical-series-quotes",
            params={
                "source": api_key,
                "symbols": symbol,
                "dayCount": (pd.Timestamp.today() - pd.Timestamp(start_date)).days,
            },
        )
    df = (
        pd.DataFrame(
            response.json()["data"]["items"][0]["historicalSeries"][
                "historicalQuoteData"
            ]
        )
        .assign(date=lambda df: df["date"].pipe(pd.to_datetime))
        .set_index("date")[::-1]
    )
    time.sleep(1)
    df.to_csv(f"./data/{name}.csv")

Retrying iEdge S-REIT Leaders SGD Gross...
Retrying iEdge S-REIT Leaders SGD Gross...
Retrying iEdge S-REIT Leaders SGD Gross...
Retrying iEdge S-REIT Leaders SGD Gross...
Retrying iEdge S-REIT Leaders SGD Gross...
Retrying iEdge S-REIT Leaders SGD Gross...
Retrying iEdge S-REIT Leaders SGD Gross...
